## Mapping human gait using unsupervised AI
This notebook demonstrates how to use the `gait_mapper` package to train a variational autoencoder for mapping human gait.

In [1]:
import numpy as np
import gait_mapper
import gait_mapper.vae

## 1 - Create the variational autoencoder
Call the `vae` module from `gait_mapper` and create the encoder and decoder network.

In [2]:
# define parameters for the model
window_length = 200
degree_of_freedom = 6
latent_features = 6

vae = gait_mapper.vae.VAE(window_length, degree_of_freedom, latent_features)

2022-02-25 11:18:28.520165: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-25 11:18:28.520601: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-25 11:18:28.524824: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
# overview of encoder network
vae.encoder.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 6)]     0                                            
__________________________________________________________________________________________________
conv1 (Conv1D)                  (None, 196, 64)      1984        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 98, 64)       0           conv1[0][0]                      
__________________________________________________________________________________________________
conv2 (Conv1D)                  (None, 96, 64)       12352       max_pooling1d[0][0]              
______________________________________________________________________________________________

In [4]:
# overview of decoder network
vae.decoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 6)]               0         
_________________________________________________________________
dense (Dense)                (None, 64)                448       
_________________________________________________________________
reshape (Reshape)            (None, 1, 64)             0         
_________________________________________________________________
conv1d_transpose (Conv1DTran (None, 3, 16)             3088      
_________________________________________________________________
conv1d_transpose_1 (Conv1DTr (None, 7, 32)             2592      
_________________________________________________________________
up_sampling1d (UpSampling1D) (None, 35, 32)            0         
_________________________________________________________________
conv1d_transpose_2 (Conv1DTr (None, 39, 64)            1030

## 2 - Loading the dataset